In [ ]:
import base64
import json
from google.cloud import aiplatform
from google.protobuf import struct_pb2
import typing
import numpy as np

### Delete datapoints

In [ ]:
with open("./metadata.json", "r") as f:
    file = f.readlines()

datapoints = []
for line in file:
    datapoints.append(
        json.loads(line)
    )

datapoints = json.dumps({
    "datapoint_ids": [d for d in datapoints]
})

In [ ]:
!curl -X POST\
-H "Content-Type: application/json" \
-H "Authorization: Bearer `gcloud auth print-access-token`" \
"https://us-central1-aiplatform.googleapis.com/v1/projects/rl-llm-dev/locations/us-central1/indexes/7799676002698788864:removeDatapoints" \
-d '{"datapoint_ids": ["0099c35e4078761606d490c55eef2772", "54cbb435a80a5fa235cc816a21932724", "2417a1d47efb57150307cfa1eddeced4", "30ab1b8b64f2ce0caec403bd8551fd58", "8d3650c9d29868c93aa635f19d1a06f0", "14160c203e8b0b843f93a093630574b6", "49554f6b8744ca58a70262b7f6fe44bb", "4da0557fca4bc721c29de46b7e6ba8c7", "a343fd32d4c58bed05dc25cf37a6fb76", "e372412d43020526115aee3efab41626"]}'

In [ ]:
!curl -X GET\
-H "Content-Type: application/json" \
-H "Authorization: Bearer `gcloud auth print-access-token`" \
"https://us-central1-aiplatform.googleapis.com/v1/projects/rl-llm-dev/locations/us-central1/indexes/7799676002698788864"

In [ ]:
! curl -X GET \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    "https://us-central1-aiplatform.googleapis.com/v1/projects/rl-llm-dev/locations/us-central1/indexEndpoints"

# Upsert with cURL

In [ ]:
!curl \
-H "Content-Type: application/json" \
-H "Authorization: Bearer `gcloud auth print-access-token`" \
"https://us-central1-aiplatform.googleapis.com/v1/projects/rl-llm-dev/locations/us-central1/indexes/7799676002698788864:upsertDatapoints" \
-d @metadata.json

# Create Index

In [ ]:
import argparse
import json
import time

from google.cloud import aiplatform_v1beta1 as aiplatform
from google.protobuf import struct_pb2
from google.protobuf.json_format import ParseDict

In [ ]:
def create_vector_index(
        project_id: str,
        location: str,
        display_name: str,
        description: str,
        metadata_schema_uri: str,
        metadata: struct_pb2.Value,
        index_update_method: aiplatform.Index.IndexUpdateMethod
):
    index_client = aiplatform.IndexServiceClient()
    index = aiplatform.Index()
    index.display_name = display_name
    index.description = description
    index.metadata_schema_uri = metadata_schema_uri
    index.metadata = metadata
    index.index_update_method = index_update_method
    
    request = aiplatform.CreateIndexRequest(
        parent=f"projects/{project_id}/locations/{location}",
        index=index
    )

    op = index_client.create_index(request=request)
    op.result()

    return op.metadata

In [ ]:
project_id = "rl-llm-dev"
location = "us-central1"
display_name = "csm-multimodal-vector-search"
description = "CSM Multimodal Vector Search"
metadata = {
    "contentsDeltaUri": "gs://csm-solution-dataset/metadata/vector_metadata.json",
    "config":{
        "dimensions": 1408,
        "approximateNeighborsCount": 150,
        "distanceMeasureType": "DOT_PRODUCT_DISTANCE",
        "featureNormType": "UNIT_L2_NORM",
        "algorithmConfig": {
            "treeAhConfig": {
                "leafNodeEmbeddingCount": 1000, 
                "fractionLeafNodesToSearch": 0.05
            }
        }
    }
}

struct = struct_pb2.Struct()
ParseDict(metadata, struct)
schema_value = struct_pb2.Value(struct_value=struct)

index_update_method = aiplatform.Index.IndexUpdateMethod(2)
metadata_schema_uri = "gs://google-cloud-aiplatform/schema/matchingengine/metadata/nearest_neighbor_search_1.0.0.yaml"

In [ ]:
create_vector_index(
    project_id=project_id,
    location=location,
    display_name=display_name,
    description=description,
    metadata_schema_uri=metadata_schema_uri,
    metadata=schema_value,
    index_update_method=index_update_method
)

In [ ]:
client = aiplatform.MetadataServiceClient()

In [ ]:
client.list_metadata_stores()

In [ ]:
from google.cloud import aiplatform_v1beta1

def sample_list_metadata_schemas():
    # Create a client
    client = aiplatform_v1beta1.MetadataServiceClient(
        client_options={
            "api_endpoint": "us-central1-aiplatform.googleapis.com"
        }
    )

    # Initialize request argument(s)
    request = aiplatform_v1beta1.ListMetadataSchemasRequest(
        parent="projects/rl-llm-dev/locations/us-central1/metadataStores/default",
    )

    # Make the request
    page_result = client.list_metadata_schemas(request=request)

    # Handle the response
    for response in page_result:
        print(response)

In [ ]:
sample_list_metadata_schemas()

In [ ]:
from google.cloud import aiplatform_v1 as aiplatform
def sample_list_metadata_stores():
    # Create a client
    client = aiplatform.MetadataServiceClient(
        client_options={
            "api_endpoint": "us-central1-aiplatform.googleapis.com"
        }
    )

    # Initialize request argument(s)
    request = aiplatform.ListMetadataStoresRequest(
        parent="projects/rl-llm-dev/locations/us-central1",
    )

    # Make the request
    page_result = client.list_metadata_stores(request=request)

    # Handle the response
    for response in page_result:
        print(response)

In [ ]:
sample_list_metadata_stores()

In [ ]:
import json

with open("./dataset/vector_metadata.json") as f:
    vecto_upsert = json.loads(f.read())

In [ ]:
vecto_upsert.keys()

In [ ]:
new_vector = [
    {"id": i["datapoint_id"],
     "embedding": i["feature_vector"]} for i in vecto_upsert["datapoints"]]

In [ ]:
with open("./dataset/vector_create_index_data.json", "w") as f:
    for i in new_vector:
        f.write(json.dumps(i) + "\n")

In [ ]:
from google.cloud import aiplatform_v1beta1 as aiplatform

In [ ]:
client = aiplatform.IndexServiceClient(
    client_options={
        "api_endpoint": "us-central1-aiplatform.googleapis.com"
    }
)
request = aiplatform.ListIndexesRequest(
    parent="projects/rl-llm-dev/locations/us-central1"
)

results = client.list_indexes(request=request)

In [ ]:
for i in list(results):
    if i.display_name == "csm-multimodal-vector-search":
        print(i.name)

In [ ]:
client = aiplatform.IndexEndpointServiceClient(
    client_options={
        "api_endpoint": "us-central1-aiplatform.googleapis.com"
    }
)

request = aiplatform.ListIndexEndpointsRequest(
    parent = f"projects/rl-llm-dev/locations/us-central1",
)

results = client.list_index_endpoints(request=request)

In [ ]:
results

In [ ]:
endpoint_name = ""
for i in list(results):
    if i.display_name == "csm-index-endpoint":
        endpoint_name = i.name
        deployed_index_id = i.deployed_indexes[0].id
        public_endpoint_domain_name = i.public_endpoint_domain_name

In [ ]:
print(endpoint_name)
print(deployed_index_id)
print(public_endpoint_domain_name)

In [1]:
import utils_toml

In [2]:
utils_toml.update_toml(
    toml_path="config.toml",
    new_values={"$$project_id$$": '"rl-llm-dev"'}
)

In [ ]:
with open("config.toml", "r+") as f:
    toml = f.read()

In [ ]:
new_values = {"$$project_id$$": "rl-llm-dev"}
for key, value in new_values.items():
    toml = toml.replace(key, value)

In [ ]:
print(toml)

In [ ]:
with open(toml_path, "w") as f:
    f.write(toml)